### M&uuml;hle, Skizze

M&uuml;hle-Koordinaten: `(r, i)`, wobei
- `r` ist einer der Ringe 0, 1 ,2.
- `i` ist einer der Punkte auf diesem Ring, nummeriert z.B.  
$\begin{array}{ccc}
6& 5& 4\\
7 &&  3\\
0 &1 &2\\
\end{array}$  


Man kann von `(r,i)` nach  `(s,j)` ziehen, falls
- `r == s and abs(i - j) in (1, 7)` (benachbart im gleichen Ring)
- `abs(r - s) == 1 and i == j and is_odd(i)` (benachbart auf Verbindung zw. Ringen)

M&uuml;hlen (falls $8\equiv 0$)
- $[(r, i), (r, i+1), (r, i+2)]$ ist M&uuml;hle, falls $r\in\{0,1,2\}$ und $i$ gerade.
- $[(0, i), (1, i), (2, i)]$ ist M&uuml;hle, falls $i$ ungerade.

In [ ]:
aus muhle nehmen falls nur muehlen!

In [301]:
class Game:
    def __init__(self):
        self.players = [0, 1]
        self.muehlen = ([[(r, i), (r, i+1), (r, (i+2) % 8)] 
                         for r in range(3) for i in range(8) if self.is_even(i)
                        ] +
                        [[(0, i), (1, i), (2, i)] for i in range(8) if self.is_odd(i)]
                       )
        self.callback = lambda: print(self)
        self.new_game()
        
    def new_game(self):
        self.board = {(r, i): None for r in range(3) for i in range(8)}
        self.ptm = 0
        self.ply = 0
        self.muehle = False
        self.result = None
       
    def is_even(self, i):
        return i % 2 == 0
    
    def is_odd(self, i):
        return i % 2 == 1
    
    def is_adjacent(self, pos1, pos2):
        r, i = pos1
        s, j = pos2
        return (r == s and abs(i - j) in (1, 7) 
                or abs(r - s) == 1 and i == j and self.is_odd(i)
               )
                
    def is_blocked(self, pos):
        return None not in set(p for pos_, p  in self.board.items() if self.is_adjacent(pos, pos_))
                
    def cannot_move(self, player):
        stones = [stone for stone, p in self.board.items() if p == player]
        return stones and all(self.is_blocked(stone) for stone in  stones)   
    
    def get_nstones(self, player):
        return list(self.board.values()).count(player)
        
    def is_muehle(self, stone):
        for muehle in self.muehlen:
            if stone in muehle and len(set(self.board[pos] for pos in muehle)) == 1:
                return True
                
    def update_result(self):
        opponent = 1 - self.ptm
        if self.cannot_move(opponent) or (self.ply >= 18 and self.get_nstones(opponent) < 3):
            self.result = self.ptm
   
    def is_legal(self, player, tp, src, target=None):
        # Spieler nicht am Zug
        if player != self.ptm:
            return
        # Muehle, aber kein Stein wird entfernt
        if self.muehle and tp != 'r':
            return 
        
        if tp == 'r':
            # keine Muehle
            if not self.muehle:
                return
            # kein gegnerischer Stein an Position src
            opponent = 1 - self.ptm
            if self.board[src] != opponent: 
                return
            
            # Stein in Muhle und Gegner hat mehr als 3 Steine
            if self.is_muehle(src) and self.get_nstones(opponent) > 3:
                return
            
        # nicht mehr in Setzphase oder Platz nicht frei    
        if tp == 'p' and (self.ply >= 18 or self.board[src] is not None):
            return
        
      
        if tp == 'm':
            # nicht in Setzphase
            if self.ply < 18:
                return
            # Platz nicht frei 
            if self.board[src] is not None:
                return
            # src und taret nicht benachbart und mehr als 3 Steine
            if self.get_nstones(self.ptm) > 3 and not self.is_adjacent(src, target):
                return
            
        return True
        
    def move(self, player, tp, src, target=None):
        if not self.is_legal(player, tp, src, target):
            return
        
        if tp == 'p':
            self.board[src] = player
            self.muehle = self.is_muehle(src)
            print('Muehle',self.muehle)
        elif tp == 'r':
            self.board[src] = None
            self.muehle = False
        elif tp == 'm':
            self.board[src] = None
            self.board[target] = player
        
        self.update_result()
        
        if self.result is None and not self.muehle:
            self.ptm = 1 - self.ptm   
            self.ply += 1
            
        self.callback()
        
    def __repr__(self):
        return 'Am Zug: {}\nResult: {}\nBoard {}'.\
                format(self.ptm, self.result, self.board)

In [302]:
game = Game()
game

Am Zug: 0
Result: None
Board {(0, 0): None, (0, 1): None, (0, 2): None, (0, 3): None, (0, 4): None, (0, 5): None, (0, 6): None, (0, 7): None, (1, 0): None, (1, 1): None, (1, 2): None, (1, 3): None, (1, 4): None, (1, 5): None, (1, 6): None, (1, 7): None, (2, 0): None, (2, 1): None, (2, 2): None, (2, 3): None, (2, 4): None, (2, 5): None, (2, 6): None, (2, 7): None}

In [289]:
game.is_legal(0, 'p', (0, 0))

True

In [300]:
game.cannot_move(1)

True

In [297]:
game.move(0, 'p', (0, 0))

Muehle None
Am Zug: 0
Result: 0
Board {(0, 0): 0, (0, 1): None, (0, 2): None, (0, 3): None, (0, 4): None, (0, 5): None, (0, 6): None, (0, 7): None, (1, 0): None, (1, 1): None, (1, 2): None, (1, 3): None, (1, 4): None, (1, 5): None, (1, 6): None, (1, 7): None, (2, 0): None, (2, 1): None, (2, 2): None, (2, 3): None, (2, 4): None, (2, 5): None, (2, 6): None, (2, 7): None}


In [243]:
game.move(1, 'p', (0, 1))

Am Zug: 0
Result: None
Board {(0, 0): 0, (0, 1): 1, (0, 2): None, (0, 3): None, (0, 4): None, (0, 5): None, (0, 6): None, (0, 7): None, (1, 0): None, (1, 1): None, (1, 2): None, (1, 3): None, (1, 4): None, (1, 5): None, (1, 6): None, (1, 7): None, (2, 0): None, (2, 1): None, (2, 2): None, (2, 3): None, (2, 4): None, (2, 5): None, (2, 6): None, (2, 7): None}


In [188]:
game.muehlen

[[(0, 0), (0, 1), (0, 2)],
 [(0, 2), (0, 3), (0, 4)],
 [(0, 4), (0, 5), (0, 6)],
 [(0, 6), (0, 7), (0, 0)],
 [(1, 0), (1, 1), (1, 2)],
 [(1, 2), (1, 3), (1, 4)],
 [(1, 4), (1, 5), (1, 6)],
 [(1, 6), (1, 7), (1, 0)],
 [(2, 0), (2, 1), (2, 2)],
 [(2, 2), (2, 3), (2, 4)],
 [(2, 4), (2, 5), (2, 6)],
 [(2, 6), (2, 7), (2, 0)],
 [(0, 1), (1, 1), (2, 1)],
 [(0, 3), (1, 3), (2, 3)],
 [(0, 5), (1, 5), (2, 5)],
 [(0, 7), (1, 7), (2, 7)]]

In [190]:
game.move(0, 'r', (0, 1))

In [192]:
game.move(0, 'm', (0, 0), (0, 1))

In [126]:
from ipycanvas import MultiCanvas
from ipywidgets import Output
out = Output(layout = {'border': '1px solid black'})


class View:
    
    width = 200
    height = 200
    
    colors = ['red', 'blue']
    line_width = 2
    radius = 7
    
    scale = 0.8
    pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
    
    def __init__(self, game):
        self.game = game
        self.mcanvas = MultiCanvas(3, width = self.width, height = self.height, 
                layout = {'border' : '1px solid black'},
               )
        self.bg, *self.p_layers = self.mcanvas
        self.bg.line_width = self.line_width
        for layer, color in zip(self.p_layers, self.colors):
            layer.fill_style = color
       
    def ri2xy(self, r, i):
        s, w, h = self.scale, self.width, self.height
        x, y = self.pts[i]
        x = (s/3*(r+1)*x + 1) * w/2
        y = (s/3*(r+1)*y + 1) * h/2
       
        return x, y    
        
    def draw_board(self):
        for muehle in self.game.muehlen:
            line = [self.ri2xy(r, i) for r,i in muehle]
            self.bg.stroke_lines(line)
            
    def draw_position(self):
        board = self.game.board
        for pos, player in board.items():
            if player is not None:
                x, y = self.ri2xy(*pos)
                self.p_layers[player].fill_circle(x, y, self.radius)
        
    def _ipython_display_(self):
        display(self.mcanvas)

In [127]:
view = View(game)
view

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [128]:
view.draw_board()

In [129]:
view.draw_position()

In [303]:
from get_closest import get_closest
from ipycanvas import MultiCanvas
from ipywidgets import Output
out = Output(layout = {'border': '1px solid black'})

class View:
    
    width = 200
    height = 200
    
    colors = ['red', 'blue']
    line_width = 2
    radius = 7
    
    scale = 0.8
    pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
    
    def __init__(self, game):
        self.game = game
        self.game.callback = self.update
        
        self.mcanvas = MultiCanvas(3, width = self.width, height = self.height, 
                layout = {'border' : '1px solid black'},
               )
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
        
        self.bg, *self.p_layers = self.mcanvas
        self.bg.line_width = self.line_width
        for layer, color in zip(self.p_layers, self.colors):
            layer.fill_style = color
            
        self.ris = [(r,i) for r in range(3) for i in range(8)]
        self.xys = [self.ri2xy(r, i) for r in range(3) for i in range(8)]
       
        self.draw_board()
       
    def ri2xy(self, r, i):
        s, w, h = self.scale, self.width, self.height
        x, y = self.pts[i]
        x = (s/3*(r+1)*x + 1) * w/2
        y = (s/3*(r+1)*y + 1) * h/2
       
        return x, y    
        
    def draw_board(self):
        for muehle in self.game.muehlen:
            line = [self.ri2xy(r, i) for r,i in muehle]
            self.bg.stroke_lines(line)
            
    def draw_position(self):
        for layer in self.p_layers:
            layer.clear()
        board = self.game.board
        for pos, player in board.items():
            if player is not None:
                x, y = self.ri2xy(*pos)
                self.p_layers[player].fill_circle(x, y, self.radius)
                
                
                
    @out.capture()        
    def on_mouse_down(self, x, y):
        idx = get_closest(self.xys, (x, y))
        if idx is not None:
            self.selected_idx = idx
            src = self.ris[idx]
            ptm = self.game.ptm
            if self.game.board[src] is not None:
                self.game.move(ptm, 'r',src)
                print(ptm, 'remove', src)
            else:
                self.game.move(ptm, 'p', src)
                print(ptm, 'place', src)
        
    @out.capture()   
    def on_mouse_up(self, x, y):   
        if self.selected_idx is None:
            return
        idx= get_closest(self.xys, (x, y))
        if idx is not None:
            target = self.ris[idx]
            src = self.ris[self.selected_idx]
            ptm = self.game.ptm
            self.game.move(ptm, 'm', src, target)
            print(ptm, src, target)
           
        self.selected_idx = None 
            
    def update(self):
        self.draw_position()
        
    def _ipython_display_(self):
        display(self.mcanvas, out)

In [304]:
game=Game()
view=View(game)
view

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [273]:
out.clear_output()


In [305]:
game.ptm

0

In [278]:
game.cannot_move(0)

True

In [280]:
game.result